In [3]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [4]:
abstract_paper = pd.read_csv('abstract_paper_for_gene.csv')

/home/nlpuser/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
full_gene_df = pd.read_csv('full_gene_df.csv')

In [6]:
abstract_paper.dropna(inplace=True)

In [7]:
abstract_paper.describe()

,metaId
count,314681.000000
mean,419505.129468
std,281928.354253
min,1.000000
25%,111742.000000
50%,570743.000000
75%,667865.000000
max,764546.000000


In [8]:
gene_list = full_gene_df['approved_symbol'].tolist()

In [9]:
abstract_paper['Keyword1'] = abstract_paper['abstract'].apply(lambda x: ', '.join(set(x.split()).intersection(set(gene_list))))

In [10]:
abstract_paper[abstract_paper['Keyword1'] != '']

,title,doi,abstract,metaId,Keyword1
782,Common Variants in CDKN2B-AS1 Associated with ...,10.1371/journal.pone.0033389,"BACKGROUND: To date, only a small portion of t...",783,CDKN2B-AS1
2602,Specific and Novel microRNAs Are Regulated as ...,10.3389/fmicb.2017.00270,"Within the last two decades, the incidence of ...",2603,"miR-212-5p, miR-132-5p"
2908,Mir-21 Mediates the Inhibitory Effect of Ang (...,10.1038/s41598-017-13305-3,MicroRNA-21 (mir-21) induced by angiotensin II...,2909,mir-21
4018,Long non-coding RNA MEG3 attends to morphine-m...,10.1080/13880209.2019.1651343,Context: Morphine is an alkaloid isolated from...,4019,MEG3
4418,Identification of cellular microRNA miR-188-3p...,10.1186/s12985-020-1283-9,BACKGROUND: Influenza A virus (IAV) continues ...,4419,miR-188-3p
...,...,...,...,...,...
757953,Predicted therapeutic targets for COVID-19 dis...,10.1016/j.imu.2020.100407,The SARS-CoV-2 causes severe pulmonary infecti...,757954,hsa-miR-1307-3p
760777,Decreased serum levels of the inflammaging mar...,10.1016/j.mad.2020.111413,Current COVID-19 pandemic poses an unprecedent...,760778,miR-146a-5p
760883,Evaluation of expression of VDR-associated lnc...,10.1186/s12879-021-06248-8,BACKGROUND: Coronavirus disease 2019 (COVID-19...,760884,"SNHG16, SNHG6"
764183,Transcriptomic Analysis Reveals Host miRNAs Co...,10.3390/microorganisms9030665,Ebola virus is a continuing threat to human po...,764184,"hsa-miR-122-5p, hsa-miR-125b-5p"


In [11]:
disease_list = pd.read_csv('disease_found_list.csv')

In [12]:
disease_list.head()

,disease
0,disease
1,influenza
2,asthma
3,respiratory failure
4,pulmonary hypertension


In [13]:
disease_set = disease_list.disease.tolist()

In [14]:
abstract_paper['Keyword2'] = abstract_paper['abstract'].apply(lambda x: ', '.join(set(x.split()).intersection(set(disease_set))))

In [25]:
gene_disease_abstract = abstract_paper[(abstract_paper['Keyword2'] != '') & (abstract_paper['Keyword1'] != '')]

In [40]:
gene_disease_abstract.tail()

,title,doi,abstract,metaId,Keyword1,Keyword2
757953,Predicted therapeutic targets for COVID-19 dis...,10.1016/j.imu.2020.100407,The SARS-CoV-2 causes severe pulmonary infecti...,757954,hsa-miR-1307-3p,"COVID-19, disease"
760777,Decreased serum levels of the inflammaging mar...,10.1016/j.mad.2020.111413,Current COVID-19 pandemic poses an unprecedent...,760778,miR-146a-5p,COVID-19
760883,Evaluation of expression of VDR-associated lnc...,10.1186/s12879-021-06248-8,BACKGROUND: Coronavirus disease 2019 (COVID-19...,760884,"SNHG16, SNHG6","syndrome, COVID-19, disease"
764183,Transcriptomic Analysis Reveals Host miRNAs Co...,10.3390/microorganisms9030665,Ebola virus is a continuing threat to human po...,764184,"hsa-miR-122-5p, hsa-miR-125b-5p","lymphopenia, disease"
764236,miRNA-200c-3p is crucial in acute respiratory ...,10.1038/celldisc.2017.21,Influenza infection and pneumonia are known to...,764237,miR-200c-3p,"influenza, syndrome, pneumonia"


In [65]:
gene_disease_abstract_map = gene_disease_abstract.assign(Keyword2=gene_disease_abstract.Keyword2.str.split(',')).explode(['Keyword2'])

In [67]:
gene_disease_abstract_map = gene_disease_abstract_map.assign(Keyword1=gene_disease_abstract.Keyword1.str.split(',')).explode(['Keyword1'])

In [71]:
gene_disease_abstract_map.reset_index(inplace=True)

In [75]:
gene_disease_abstract_map = gene_disease_abstract_map.rename(columns={'Keyword1': 'gene', 'Keyword2': 'disease'})

In [79]:
gene_disease_abstract_map = gene_disease_abstract_map.drop(columns=['index'], axis=1)

In [82]:
gene_disease_abstract_map.dropna(subset=['gene', 'disease'])

,title,doi,abstract,metaId,gene,disease
0,Common Variants in CDKN2B-AS1 Associated with ...,10.1371/journal.pone.0033389,"BACKGROUND: To date, only a small portion of t...",783,CDKN2B-AS1,glaucoma
1,The lncRNA Malat1 functions as a ceRNA to cont...,10.1038/s41401-019-0284-y,Long non-coding RNAs (lncRNAs) have been ident...,12926,miR-181c-5p,adenocarcinoma
2,No evidence for sylvatic cycles of chikungunya...,10.1186/s13071-020-04419-1,"BACKGROUND: Dengue, chikungunya and Zika virus...",13550,PRNT,chikungunya
3,Clinical implications and nomogram prediction ...,10.1097/md.0000000000022806,"Colorectal cancer, especially colon adenocarci...",13556,FRGCA,adenocarcinoma
4,Droplet Digital PCR Analysis of Liquid Biopsy ...,10.3390/biology9110379,SIMPLE SUMMARY: Despite the availability of sc...,14242,hsa-miR-375-3p,cancer
...,...,...,...,...,...,...
407,Transcriptomic Analysis Reveals Host miRNAs Co...,10.3390/microorganisms9030665,Ebola virus is a continuing threat to human po...,764184,hsa-miR-122-5p,disease
408,Transcriptomic Analysis Reveals Host miRNAs Co...,10.3390/microorganisms9030665,Ebola virus is a continuing threat to human po...,764184,hsa-miR-125b-5p,disease
409,miRNA-200c-3p is crucial in acute respiratory ...,10.1038/celldisc.2017.21,Influenza infection and pneumonia are known to...,764237,miR-200c-3p,influenza
410,miRNA-200c-3p is crucial in acute respiratory ...,10.1038/celldisc.2017.21,Influenza infection and pneumonia are known to...,764237,miR-200c-3p,syndrome


In [83]:
gene_disease_abstract_map.to_csv('gene_disease_abstract_map.csv', index=False)